# Data Cleaning and Preparation

Source of dataset: https://www.kaggle.com/datasets/jocelyndumlao/cardiovascular-disease-dataset

Objective:
- Remove any rows with missing or duplicate data (if necessary)
- Remove irrelevant columns from analysis
- Split categorical data into separate one-hot columns

## Initial steps

### Importing libraries and initializations

In [1]:
import os

import pandas as pd
from sklearn.preprocessing import OneHotEncoder

### Import data set

In [2]:
data = pd.read_csv(os.path.join("data", "Cardiovascular_Disease_Dataset.csv"))

data.head()

,patientid,age,gender,chestpain,restingBP,serumcholestrol,fastingbloodsugar,restingrelectro,maxheartrate,exerciseangia,oldpeak,slope,noofmajorvessels,target
0,103368,53,1,2,171,0,0,1,147,0,5.3,3,3,1
1,119250,40,1,0,94,229,0,1,115,0,3.7,1,1,0
2,119372,49,1,2,133,142,0,0,202,1,5.0,1,0,0
3,132514,43,1,0,138,295,1,1,153,0,3.2,2,2,1
4,146211,31,1,1,199,0,0,2,136,0,5.3,3,2,1


#### Overview of dataset

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patientid          1000 non-null   int64  
 1   age                1000 non-null   int64  
 2   gender             1000 non-null   int64  
 3   chestpain          1000 non-null   int64  
 4   restingBP          1000 non-null   int64  
 5   serumcholestrol    1000 non-null   int64  
 6   fastingbloodsugar  1000 non-null   int64  
 7   restingrelectro    1000 non-null   int64  
 8   maxheartrate       1000 non-null   int64  
 9   exerciseangia      1000 non-null   int64  
 10  oldpeak            1000 non-null   float64
 11  slope              1000 non-null   int64  
 12  noofmajorvessels   1000 non-null   int64  
 13  target             1000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 109.5 KB


### Helper class for tracking column names

In [4]:
class Columns():
    M_PATIENT_ID = "patientid"
    N_AGE = "age"
    B_GENDER = "gender"
    C_CHEST_PAIN = "chestpain"
    N_RESTING_BP = "restingBP"
    N_SERUM_CHOLESTROL = "serumcholestrol"
    B_FASTING_BLOOD_SUGAR = "fastingbloodsugar"
    C_RESTING_R_ELECTRO = "restingrelectro"
    N_MAX_HEART_RATE = "maxheartrate"
    B_EXERCISE_ANGIA = "exerciseangia"
    N_OLD_PEAK = "oldpeak"
    C_SLOPE = "slope"
    N_NO_OF_MAJOR_VESSELS = "noofmajorvessels"
    B_TARGET = "target" 

## Removing errors, duplicates and unnecessary values from dataset

### Checking for duplicated entries

We can check for duplicated entries by comparing the number of unique patient IDs with the number of rows. If the values are not equal, then it is certain that some entries are duplicated.

In [5]:
row_cnt = len(data)
patient_id_cnt = data[Columns.M_PATIENT_ID].nunique()

print("Number of rows: ", row_cnt)
print("Number of unique patient IDs: ", patient_id_cnt)
print("No duplicate entries: ", row_cnt == patient_id_cnt)


Number of rows:  1000
Number of unique patient IDs:  1000
No duplicate entries:  True


No duplicate entries are in the dataset. So, we can move forward without having to drop any rows

### Checking for missing values

In [6]:
data.isna().sum()

patientid            0
age                  0
gender               0
chestpain            0
restingBP            0
serumcholestrol      0
fastingbloodsugar    0
restingrelectro      0
maxheartrate         0
exerciseangia        0
oldpeak              0
slope                0
noofmajorvessels     0
target               0
dtype: int64

It is clear that there are no missing entries, so we can move forward without having to drop any rows

### Dropping irrelevant rows

Certain rows will not be helpful for our analysis. We will drop these columns from our table.

From reading the descriptions of the columns in the pdf file associated with this dataset, we can see the patient ID column will not be helpful for our analysis.

In [7]:
data.drop(Columns.M_PATIENT_ID, axis=1, inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                1000 non-null   int64  
 1   gender             1000 non-null   int64  
 2   chestpain          1000 non-null   int64  
 3   restingBP          1000 non-null   int64  
 4   serumcholestrol    1000 non-null   int64  
 5   fastingbloodsugar  1000 non-null   int64  
 6   restingrelectro    1000 non-null   int64  
 7   maxheartrate       1000 non-null   int64  
 8   exerciseangia      1000 non-null   int64  
 9   oldpeak            1000 non-null   float64
 10  slope              1000 non-null   int64  
 11  noofmajorvessels   1000 non-null   int64  
 12  target             1000 non-null   int64  
dtypes: float64(1), int64(12)
memory usage: 101.7 KB


## Splitting categorical variables into one-hot columns

### Identifying relevant variables

We can identify the columns which needs to be split into multiple one-hot columns from the pdf file:
1. **chestpain** (Chest pain type):
  - 0: Typical Angina
  - 1: Atypical Angina
  - 2: Non-anginal pain
  - 3: Asymptomatic
2. **restingrelectro** (Resting electrogram results):
  - 0: Normal
  - 1: Having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of >0.05 mV)
  - 2: Showing probable or definite left ventricular hypertrophy (by Estes' criteria)
3. **slope** (Slope of the peak exercise ST segment):
  - 1: Upsloping
  - 2: Flat
  - 3: Downsloping

In [8]:
data_cat = data[[Columns.C_CHEST_PAIN,Columns.C_RESTING_R_ELECTRO, Columns.C_SLOPE]]

print("Unique values for column chestpain: ", data_cat[Columns.C_CHEST_PAIN].unique())
print("Unique values for column restingrelectro: ", data_cat[Columns.C_RESTING_R_ELECTRO].unique())
print("Unique values for column slope: ", data_cat[Columns.C_SLOPE].unique())

Unique values for column chestpain:  [2 0 1 3]
Unique values for column restingrelectro:  [1 0 2]
Unique values for column slope:  [3 1 2 0]


Notice that for column slope there is an extra category of 0 which is undefined in the original pdf file.

In [9]:
len(data_cat[data_cat[Columns.C_SLOPE] == 0])

180

With 180 entries, we can assume that this is not a mistake in the original dataset, but just a category which was not described in the given pdf file. We can move forward by simply labeling this category as *Undefined*.

### Replacing enumerations with short descriptive names for one-hot encoding later

In [10]:
data_cat = data_cat.replace({
    Columns.C_CHEST_PAIN:{
        0: "TypicalAngina",
        1: "AtypicalAngina",
        2: "NonAnginal",
        3: "Asymptomatic",
    },
    Columns.C_RESTING_R_ELECTRO:{
        0: "Normal",
        1: "WaveAbnormality",
        2: "LeftVentricularHypertrophy",
    },
    Columns.C_SLOPE:{
        0: "Undefined",
        1: "Upsloping",
        2: "Flat",
        3: "Downsloping",
    }}
)

data_cat.head()

,chestpain,restingrelectro,slope
0,NonAnginal,WaveAbnormality,Downsloping
1,TypicalAngina,WaveAbnormality,Upsloping
2,NonAnginal,Normal,Upsloping
3,TypicalAngina,WaveAbnormality,Flat
4,AtypicalAngina,LeftVentricularHypertrophy,Downsloping


### One-hot encode the identified columns

In [11]:
ohe = OneHotEncoder()

ohe.fit(data_cat)
data_cat_ohe = pd.DataFrame(
    ohe.transform(data_cat).toarray(), 
    columns=ohe.get_feature_names_out(data_cat.columns)
)

data_cat_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   chestpain_Asymptomatic                      1000 non-null   float64
 1   chestpain_AtypicalAngina                    1000 non-null   float64
 2   chestpain_NonAnginal                        1000 non-null   float64
 3   chestpain_TypicalAngina                     1000 non-null   float64
 4   restingrelectro_LeftVentricularHypertrophy  1000 non-null   float64
 5   restingrelectro_Normal                      1000 non-null   float64
 6   restingrelectro_WaveAbnormality             1000 non-null   float64
 7   slope_Downsloping                           1000 non-null   float64
 8   slope_Flat                                  1000 non-null   float64
 9   slope_Undefined                             1000 non-null   float64
 10  slope_Upslopi

## Concatenating columns back into final data frame

In [12]:
data_numeric = pd.DataFrame(data[[
    Columns.N_AGE,
    Columns.N_RESTING_BP,
    Columns.N_SERUM_CHOLESTROL,
    Columns.N_MAX_HEART_RATE,
    Columns.N_OLD_PEAK,
    Columns.N_NO_OF_MAJOR_VESSELS,
]])
data_binary = pd.DataFrame(data[[
    Columns.B_GENDER,
    Columns.B_FASTING_BLOOD_SUGAR,
    Columns.B_EXERCISE_ANGIA,
]])
data_target = pd.DataFrame(data[Columns.B_TARGET])

### Renaming & Type conversions

#### Converting dtypes of certain columns for better clarity and reduce dataset size

In [13]:
data_binary = data_binary.astype(bool)
data_cat_ohe = data_cat_ohe.astype(bool)
data_target = data_target.astype(bool)

#### Renaming certain columns for better clarity

In [14]:
data_binary.rename(columns={Columns.B_GENDER: "isMale"}, inplace=True)
data_target.rename(columns={Columns.B_TARGET: "hasHeartDisease"}, inplace=True)

### Concatenating into final dataset

In [15]:
data_final = pd.concat([data_numeric, data_binary, data_cat_ohe, data_target], axis = 1).reindex(index=data_numeric.index)

data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   age                                         1000 non-null   int64  
 1   restingBP                                   1000 non-null   int64  
 2   serumcholestrol                             1000 non-null   int64  
 3   maxheartrate                                1000 non-null   int64  
 4   oldpeak                                     1000 non-null   float64
 5   noofmajorvessels                            1000 non-null   int64  
 6   isMale                                      1000 non-null   bool   
 7   fastingbloodsugar                           1000 non-null   bool   
 8   exerciseangia                               1000 non-null   bool   
 9   chestpain_Asymptomatic                      1000 non-null   bool   
 10  chestpain_Aty

## Export dataframe to .pkl file to use for next steps

In [16]:
data_final.to_pickle(os.path.join("data","data_postprep.pkl"))